## OpenAI Setup

In [1]:
!pip -q install openai

In [2]:
import json
import openai
import uuid
import numpy as np

#openai.api_key =  # isert valid GPT key for use

## Example Class

In [3]:
class Example:

   # Stores an input, output pair and formats it to prime the model
   def __init__(self, input, output):
       self.input = input
       self.output = output
       self.id = uuid.uuid4().hex

   # To obtain the input provided for an example
   def get_input(self):
       return self.input

   # To obtain the output provided for an example
   def get_output(self):
       return self.output

   # To obtain the unique id of an example
   def get_id(self):
       return self.id

## GPT3 Class

In [4]:
class GPT3:

   """
      Params engine: Model to be used. Options are Davinci, Babbage, Ada and Curie.
      temperature: Amount of randomness to be introduced in the predictions of the
      model Setting a higher value of temperature would be useful for creative 
      applications whereas a lower value will be suitable for well defined answers.
      max_tokens: Maximum length of number of tokens accepted by the prompt.
   """

   # initialises parameters and adds default values
   def __init__(self, engine='davinci', temperature=0.5, max_tokens=100,

       input_prefix="input: ", input_suffix="\n", output_prefix="output: ",
       output_suffix="\n\n", append_output_prefix_to_query=False):
       self.examples = {}
       self.engine = engine
       self.temperature = temperature
       self.max_tokens = max_tokens
       self.input_prefix = input_prefix
       self.input_suffix = input_suffix
       self.output_prefix = output_prefix
       self.output_suffix = output_suffix
       self.append_output_prefix_to_query = append_output_prefix_to_query
       self.stop = (output_suffix + input_prefix).strip()

   # Adds an example to the model object. Example is an instance of the Example class.
   def add_example(self, ex):
       assert isinstance(ex, Example), "Please create an Example object."
       self.examples[ex.get_id()] = ex

   # Converts all the examples to a particular format to prime the model.
   def get_prime_text(self):
       return "".join(
           [self.format_example(ex) for ex in self.examples.values()])

   # Creates a query for the API request
   def craft_query(self, prompt):
       q = self.get_prime_text(
       ) + self.input_prefix + prompt + self.input_suffix

       if self.append_output_prefix_to_query:
           q = q + self.output_prefix
       return q

   # Calls the API using the Completion endpoint with the specified values of the parameters
   def submit_request(self, prompt):
       response = openai.Completion.create(engine=self.engine,
                                           prompt=self.craft_query(prompt),
                                           max_tokens=self.max_tokens,
                                           temperature=self.temperature,
                                           top_p=1,
                                           n=1,
                                           stream=False,
                                           stop=self.stop)
       return response

   # Formats the input output pair with appropriate prefixes and suffixes
   def format_example(self, ex):
       return self.input_prefix + ex.get_input(
       ) + self.input_suffix + self.output_prefix + ex.get_output(
       ) + self.output_suffix

# Atos de Extrato de Contrato

In [5]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!ls  drive/MyDrive/Datasets/*csv 

drive/MyDrive/Datasets/dodf_atos_pessoal_v3.csv
drive/MyDrive/Datasets/dodf_atos_pessoal_v4.csv
drive/MyDrive/Datasets/DODFCorpus_contratos_licitacoes_v1.csv


### Pré-processamento

Acesso ao corpus de Contratos e Licitações:

In [8]:
df = pd.read_csv('drive/MyDrive/Datasets/DODFCorpus_contratos_licitacoes_v1.csv')

Listando tipos de ato presentes no corpus:

In [9]:
tipos_ato = df.tipo_rel.unique()
tipos_ato

array(['REL_AVISO_LICITACAO', 'REL_SUSPENSAO_LICITACAO',
       'REL_EXTRATO_CONTRATO', 'REL_ADITAMENTO_CONTRATO',
       'REL_EXTRATO_CONVENIO', 'REL_ANUL_REVOG_LICITACAO'], dtype=object)

In [10]:
df_extrato_contrato_label = df[df.tipo_rel=='REL_EXTRATO_CONTRATO']
df_extrato_contrato = df_extrato_contrato_label[df_extrato_contrato_label.tipo_ent=='EXTRATO_CONTRATO'].copy()

Transformando anotação em um dicionário...

In [11]:
labels = []
for name, group in  df_extrato_contrato_label.groupby('id_ato', sort=False):
    d = {}
    for ent, txt in zip(group.tipo_ent, group.texto):
        if ent.isupper():
             continue
        if ent not in d:
            d[ent] = [txt]
        else:
            d[ent].append(txt)
    labels.append(d)

In [12]:
df_extrato_contrato['label'] = labels
df_extrato_contrato.label.values[0]

{'numero_contrato': ['45/2018'],
 'processo_gdf': ['00070-00016600/2018-76'],
 'orgao_contratante': ['SEAGRI/DF'],
 'entidade_contratada': ['CIMAG  COMERCIO DE\nIMPLEMENTOS E MAQUINAS AGRICOLAS LTDA'],
 'valor_contrato': ['39.600,00'],
 'nota_empenho': ['2018NE00463', '2018NE00464'],
 'unidade_orcamentaria': ['14101'],
 'programa_trabalho': ['20.606.6207.2889.0003'],
 'natureza_despesa': ['449052'],
 'fonte_recurso': ['732014482', '100000000'],
 'data_assinatura_contrato': ['21/11/2018'],
 'objeto_contrato': ['O Contrato tem por objeto a aquisicao\ndos seguintes itens: Item 20: Quantidade 03 ( tres). GRADE ARADORA CONTROLE R E M O TO ,\ncom 14 discos de 26 polegadas de diametro e 06 mm de espessura minima, mancais de rolamento\ncom lubrificacao permanente em banho de oleo, ou a graxa, espacamento minimo entre os discos de\n230mm. controle remoto para regulagem de profundidade do trabalho e transporte por meio do sistema\nhidraulico e pneus agricolas largura de corte minima de 1500 mm. 

Transformando dicionários em uma string de anotação...

In [13]:
df_extrato_contrato['label_txt'] = df_extrato_contrato.label.apply(
    lambda x :' # '.join([' # '.join([f'{k}: {v}'for v in vs]) for k,vs in x.items()]))

In [14]:
df_extrato_contrato.label_txt.values[0]

'numero_contrato: 45/2018 # processo_gdf: 00070-00016600/2018-76 # orgao_contratante: SEAGRI/DF # entidade_contratada: CIMAG  COMERCIO DE\nIMPLEMENTOS E MAQUINAS AGRICOLAS LTDA # valor_contrato: 39.600,00 # nota_empenho: 2018NE00463 # nota_empenho: 2018NE00464 # unidade_orcamentaria: 14101 # programa_trabalho: 20.606.6207.2889.0003 # natureza_despesa: 449052 # fonte_recurso: 732014482 # fonte_recurso: 100000000 # data_assinatura_contrato: 21/11/2018 # objeto_contrato: O Contrato tem por objeto a aquisicao\ndos seguintes itens: Item 20: Quantidade 03 ( tres). GRADE ARADORA CONTROLE R E M O TO ,\ncom 14 discos de 26 polegadas de diametro e 06 mm de espessura minima, mancais de rolamento\ncom lubrificacao permanente em banho de oleo, ou a graxa, espacamento minimo entre os discos de\n230mm. controle remoto para regulagem de profundidade do trabalho e transporte por meio do sistema\nhidraulico e pneus agricolas largura de corte minima de 1500 mm. compativel com tratores de\npotencia minima

In [15]:
df_extrato_contrato[['id_ato', 'texto', 'label', 'label_txt']].head()

,id_ato,texto,label,label_txt
194,12_22.11.2018-R1,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'numero_contrato': ['45/2018'], 'processo_gdf...",numero_contrato: 45/2018 # processo_gdf: 00070...
210,12_22.11.2018-R2,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...,"{'orgao_contratante': ['SEAGRI/DF'], 'entidade...",orgao_contratante: SEAGRI/DF # entidade_contra...
240,12_22.11.2018-R3,EXTRATO DO CONTRATO No 32/2018\nAQUISICAO DE B...,"{'numero_contrato': ['32/2018'], 'processo_gdf...",numero_contrato: 32/2018 # processo_gdf: 00053...
297,12_22.11.2018-R4,EXTRATO DE CONTRATO\nCONTRATO No 9020. ASSINAT...,"{'numero_contrato': ['9020'], 'data_assinatura...",numero_contrato: 9020 # data_assinatura_contra...
371,12_22.11.2018-R7,"EXTRATO DO CONTRATO No 01/2018,\nNOS TERMOS DO...","{'numero_contrato': ['01/2018'], 'processo_gdf...",numero_contrato: 01/2018 # processo_gdf: 00367...


### Treinando GPT

#### Procurando exemplos para treinamento

Não existe ato contendo todas as entidades, sendo assim, é necessário procurar o menor número de atos que tenham exemplos de todas as entidades.

In [16]:
entidades_extrato_contrato = set(filter(str.islower, df_extrato_contrato_label.tipo_ent.unique()))
print('\n'.join(sorted(entidades_extrato_contrato)))
print('-'*50)
print(f'{len(entidades_extrato_contrato)} entidades')

cnpj_entidade_contratada
cnpj_orgao_contratante
codigo_siggo
data_assinatura_contrato
entidade_contratada
fonte_recurso
natureza_despesa
nome_responsavel
nota_empenho
numero_contrato
objeto_contrato
orgao_contratante
processo_gdf
programa_trabalho
unidade_orcamentaria
valor_contrato
vigencia_contrato
--------------------------------------------------
17 entidades


Organizando os atos por quantidade de entidades:

In [17]:
most_entities_ato = np.argsort([(-len(d)) for d in df_extrato_contrato.label.values])
most_entities_ato

array([1147, 1396, 1397, ..., 1227, 1422, 1433])

Selecionando atos para serem utilizados como exemplo, empregando o maior número de entidades possível

In [18]:
acc = set()
example_ids = []
for e in most_entities_ato:
    if not len(entidades_extrato_contrato - acc):
        break
    x = set(df_extrato_contrato.label.values[e])
    if len(set(x) - acc):
        acc |= set(x)
        example_ids.append(e)


In [19]:
for id in example_ids:
    ents = df_extrato_contrato.label.values[id]
    print('id:', id)
    print(f'{len(ents)} Entities:', set(ents))
    print('Missing :', entidades_extrato_contrato - set(ents))
    print('-'*50)

id: 1147
15 Entities: {'natureza_despesa', 'unidade_orcamentaria', 'entidade_contratada', 'numero_contrato', 'programa_trabalho', 'data_assinatura_contrato', 'fonte_recurso', 'objeto_contrato', 'nota_empenho', 'processo_gdf', 'orgao_contratante', 'cnpj_entidade_contratada', 'valor_contrato', 'vigencia_contrato', 'cnpj_orgao_contratante'}
Missing : {'codigo_siggo', 'nome_responsavel'}
--------------------------------------------------
id: 646
15 Entities: {'unidade_orcamentaria', 'natureza_despesa', 'entidade_contratada', 'numero_contrato', 'programa_trabalho', 'data_assinatura_contrato', 'fonte_recurso', 'objeto_contrato', 'nota_empenho', 'processo_gdf', 'orgao_contratante', 'codigo_siggo', 'cnpj_entidade_contratada', 'valor_contrato', 'vigencia_contrato'}
Missing : {'nome_responsavel', 'cnpj_orgao_contratante'}
--------------------------------------------------
id: 865
13 Entities: {'unidade_orcamentaria', 'entidade_contratada', 'numero_contrato', 'programa_trabalho', 'data_assinatura

#### Realizando treinamento com exemplos

In [20]:
input_txt = df_extrato_contrato.texto.values[example_ids]
input_txt[0]

'EXTRATO DE CONTRATO PARA AQUISIÇÃO DE BENS Nº 12/2021\nProcesso: 04011-00001803/2021-37. Partes: O DISTRITO FEDERAL, por meio da SECRETARIA DE ESTADO DA MULHER DO DISTRITO FEDERAL, CNPJ nº 15.169.975/0001-15, e a empresa INDÚSTRIA DE ÁGUA MINERAL IBIÁ LTDA, CNPJ nº 05.655.158/0001-13. Objeto: aquisição de material do gênero alimentício (água potável) e material de acondicionamento e embalagem (garrafão retornável - vasilhame) destinado ao funcionamento desta Secretaria de Estado da Mulher do Distrito Federal. UNIDADE ORÇAMENTÁRIA: 57.101. PROGRAMA DE TRABALHO: 14.122.8211.8517.0163. NATUREZA DA DESPESA: 339030. FONTE DE RECURSO: 100. NOTA DE EMPENHO INICAL: nº 2021NE00159, no valor de R$ 3.186,00 (três mil, cento e oitenta e seis reais), emitida em 24/08/2021. EVENTO: 400091. MODALIDADE: Estimativo. VALOR DO CONTRATO: R$ 11.469,60 (onze mil, quatrocentos e sessenta e nove reais e sessenta centavos). VIGÊNCIA: O contrato terá vigência de 12 (dose) meses, a contar de 03/09/2021 até 03/0

In [21]:
output_txt = df_extrato_contrato.label_txt.values[example_ids]
output_txt[0]

'fonte_recurso: 100 # natureza_despesa: 339030 # entidade_contratada: INDÚSTRIA DE ÁGUA MINERAL IBIÁ LTDA # cnpj_entidade_contratada: 05.655.158/0001-13 # numero_contrato: 12/2021 # processo_gdf: 04011-00001803/2021-37 # orgao_contratante: SECRETARIA DE ESTADO DA MULHER DO DISTRITO FEDERAL # cnpj_orgao_contratante: 15.169.975/0001-15 # objeto_contrato: aquisição de material do gênero alimentício (água potável) e material de acondicionamento e embalagem (garrafão retornável - vasilhame) destinado ao funcionamento desta Secretaria de Estado da Mulher do Distrito Federal # unidade_orcamentaria: 57.101 # programa_trabalho: 14.122.8211.8517.0163 # nota_empenho: 2021NE00159 # valor_contrato: 11.469,60 # vigencia_contrato: O contrato terá vigência de 12 (dose) meses, a contar de 03/09/2021 até 03/09/2022 # data_assinatura_contrato: 24/08/2021'

In [22]:
models = [GPT3(engine="davinci", temperature=0, max_tokens=400) for _ in range(len(example_ids))]

In [23]:
for model, i, o in zip(models, input_txt, output_txt):
    model.add_example(Example(i, o))

### Realizando Predições

In [24]:
np.random.seed(42)

In [25]:
from tqdm.notebook import tqdm

cont_tokens = 0
preds = []
for text in tqdm(df_extrato_contrato.texto):
    n = np.random.randint(0, len(example_ids))
    try:
        response =  models[n].submit_request(text)
        pred = (response.to_dict()["choices"][0]["text"].replace("output: ", ''))
        preds.append(pred)
        # contador de tokens:
        cont_tokens += int(response.to_dict()['usage']['total_tokens'])
    except Exception as e:
        print(f'Modelo: {n}')
        print(f'Erro:{e}')
        preds.append('')
print('-'*50)        
print("Numero total de tokens usadas",cont_tokens)
print("preco final:",(cont_tokens/1000)*0.02)

  0%|          | 0/1542 [00:00<?, ?it/s]

Modelo: 2
Erro:This model's maximum context length is 2049 tokens, however you requested 2144 tokens (1744 in your prompt; 400 for the completion). Please reduce your prompt; or completion length.
Modelo: 2
Erro:This model's maximum context length is 2049 tokens, however you requested 2291 tokens (1891 in your prompt; 400 for the completion). Please reduce your prompt; or completion length.
Modelo: 1
Erro:This model's maximum context length is 2049 tokens, however you requested 2070 tokens (1670 in your prompt; 400 for the completion). Please reduce your prompt; or completion length.
Modelo: 2
Erro:This model's maximum context length is 2049 tokens, however you requested 2198 tokens (1798 in your prompt; 400 for the completion). Please reduce your prompt; or completion length.
Modelo: 2
Erro:This model's maximum context length is 2049 tokens, however you requested 2393 tokens (1993 in your prompt; 400 for the completion). Please reduce your prompt; or completion length.
Modelo: 1
Erro:

Transformando predições em dicionários:

In [49]:
dict_preds = []
for pred in preds:
    dict_pred = {}
    list_pred = [(k.strip(),v.strip()) for k, _, v in  [x.partition(':') for x in pred.split('#')] if k!=v!='']
    for k, v in list_pred:
        if k not in dict_pred:
            if k in entidades_extrato_contrato:
                dict_pred[k] = [v]
        else:
            dict_pred[k].append(v)
    dict_preds.append(dict_pred)

Contando quantidade de predições feitas de fato:

In [56]:
n_errors = len([x for x in dict_preds if x!={}])
print(f'{len(preds) - n_errors} predições de {len(preds)} atos')

759 predições de 1542 atos


In [57]:
df_extrato_contrato['pred'] = dict_preds

In [59]:
df_extrato_contrato[['label', 'pred']]

,label,pred
194,"{'numero_contrato': ['45/2018'], 'processo_gdf...",{}
210,"{'orgao_contratante': ['SEAGRI/DF'], 'entidade...","{'fonte_recurso': ['332012027'], 'natureza_des..."
240,"{'numero_contrato': ['32/2018'], 'processo_gdf...",{'orgao_contratante': ['INSTITUTO DE ASSIST. À...
297,"{'numero_contrato': ['9020'], 'data_assinatura...",{}
371,"{'numero_contrato': ['01/2018'], 'processo_gdf...","{'fonte_recurso': ['100'], 'natureza_despesa':..."
...,...,...
40914,"{'data_assinatura_contrato': ['15/03/2022'], '...","{'orgao_contratante': ['SEEDF', 'SEEDF'], 'obj..."
40924,"{'orgao_contratante': ['SEEDF'], 'unidade_orca...",{}
40994,"{'fonte_recurso': ['100'], 'data_assinatura_co...",{}
41008,"{'fonte_recurso': ['100'], 'data_assinatura_co...","{'fonte_recurso': ['100'], 'natureza_despesa':..."


Salvando resultados...

In [60]:
import pickle as pkl

with open('drive/MyDrive/Datasets/preds_extrato_contrato.pkl', 'wb') as f:
    pkl.dump(preds, f)

with open('drive/MyDrive/Datasets/dict_preds_extrato_contrato.pkl', 'wb') as f:
    pkl.dump(dict_preds, f)

with open('drive/MyDrive/Datasets/df_extrato_contrato.pkl', 'wb') as f:
    pkl.dump(df_extrato_contrato, f)

In [61]:
df_extrato_contrato.to_csv('drive/MyDrive/Datasets/df_extrato_contrato.csv')